In [14]:
import os
import cv2
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, UpSampling2D

# Define the paths to the clear and foggy image folders
clear_folder = "C:/Users/maraf/OneDrive/Desktop/clearFoggy/clearFoggy/Clear/"
foggy_folder = "C:/Users/maraf/OneDrive/Desktop/clearFoggy/clearFoggy/Foggy/"

# Set the desired image size
image_height = 64
image_width = 64
num_channels = 3

# Load the clear and foggy images
clear_images = []
foggy_images = []

for filename in os.listdir(foggy_folder):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        foggy_img = cv2.imread(os.path.join(foggy_folder, filename))
        clear_filename = os.path.splitext(filename)[0] + ".jpg"  # Assuming clear images have .jpg extension
        clear_img = cv2.imread(os.path.join(clear_folder, clear_filename))
        foggy_img = cv2.resize(foggy_img, (image_width, image_height))
        clear_img = cv2.resize(clear_img, (image_width, image_height))
        foggy_images.append(foggy_img)
        clear_images.append(clear_img)

# Convert the lists of images to numpy arrays
clear_images = np.array(clear_images)
foggy_images = np.array(foggy_images)

# Normalize pixel values to range between 0 and 1
clear_images = clear_images.astype('float32') / 255.0
foggy_images = foggy_images.astype('float32') / 255.0

# Define the CNN model architecture
model = Sequential()
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', input_shape=(image_height, image_width, num_channels)))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
#model.add(UpSampling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(num_channels, (3, 3), activation='sigmoid', padding='same'))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(foggy_images, clear_images, epochs=1, batch_size=1)

# Save the trained model
model.save("foggy_to_clear_model.h5")


10/10 [==============================] - 4s 123ms/step - loss: 0.0955


In [20]:
import os
import cv2
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D
from keras.callbacks import EarlyStopping

# Define the paths to the clear and foggy image folders
clear_folder_path = "C:/Users/maraf/OneDrive/Desktop/clearFoggy/clearFoggy/Clear"
foggy_folder_path = "C:/Users/maraf/OneDrive/Desktop/clearFoggy/clearFoggy/Foggy"

# Set the desired image size
image_height = 256
image_width = 256
num_channels = 3

# Get the list of file names in the clear folder
clear_file_names = os.listdir(clear_folder_path)

# Create an empty list to store clear and foggy images
clear_images = []
foggy_images = []

# Iterate over each file name in the clear folder
for file_name in clear_file_names:
    # Construct the paths to the clear and foggy images
    clear_image_path = os.path.join(clear_folder_path, file_name)
    foggy_image_path = os.path.join(foggy_folder_path, file_name)

    # Load the clear and foggy images
    clear_img = cv2.imread(clear_image_path)
    foggy_img = cv2.imread(foggy_image_path)

    # Resize the images
    clear_img = cv2.resize(clear_img, (image_width, image_height))
    foggy_img = cv2.resize(foggy_img, (image_width, image_height))

    # Convert the images to numpy arrays
    clear_images.append(clear_img)
    foggy_images.append(foggy_img)

# Convert the lists to numpy arrays
clear_images = np.array(clear_images)
foggy_images = np.array(foggy_images)

# Normalize pixel values to range between 0 and 1
clear_images = clear_images.astype('float32') / 255.0
foggy_images = foggy_images.astype('float32') / 255.0

# Define the CNN model architecture
model = Sequential()
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(image_height, image_width, num_channels)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(num_channels, (3, 3), activation='sigmoid', padding='same'))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Define early stopping callback
early_stopping = EarlyStopping(monitor='accuracy', patience=0, min_delta=0.01, mode='max')

# Train the model
model.fit(foggy_images, clear_images, epochs=500, batch_size=32, callbacks=[early_stopping])

# Save the trained model
model.save("foggy_to_clear_model_10K.h5")


Epoch 1/500
1/1 [==============================] - 5s 5s/step - loss: 0.0973
Epoch 2/500
1/1 [==============================] - 4s 4s/step - loss: 0.0944
Epoch 3/500
1/1 [==============================] - 4s 4s/step - loss: 0.0935
Epoch 4/500
1/1 [==============================] - 4s 4s/step - loss: 0.0929
Epoch 5/500
1/1 [==============================] - 4s 4s/step - loss: 0.0922
Epoch 6/500
1/1 [==============================] - 4s 4s/step - loss: 0.0918
Epoch 7/500
1/1 [==============================] - 4s 4s/step - loss: 0.0914
Epoch 8/500
1/1 [==============================] - 4s 4s/step - loss: 0.0908
Epoch 9/500
1/1 [==============================] - 4s 4s/step - loss: 0.0901
Epoch 10/500
1/1 [==============================] - 4s 4s/step - loss: 0.0894
Epoch 11/500
1/1 [==============================] - 4s 4s/step - loss: 0.0884
Epoch 12/500
1/1 [==============================] - 4s 4s/step - loss: 0.0871
Epoch 13/500
1/1 [==============================] - 4s 4s/step - loss: 0.

In [3]:
import cv2
import numpy as np
from keras.models import load_model

# Set the desired image size
image_height = 256
image_width = 256
num_channels = 3

# Load the trained model
model = load_model("foggy_to_clear_model_10K.h5")

# Load a foggy image for testing
foggy_image_path = "C:/Users/maraf/OneDrive/Desktop/Fog/Fog/foggy-003.jpg"
foggy_image = cv2.imread(foggy_image_path)

# Resize the foggy image to the desired size
foggy_image = cv2.resize(foggy_image, (image_width, image_height))

# Preprocess the foggy image
foggy_image = foggy_image.astype('float32') / 255.0
foggy_image = np.expand_dims(foggy_image, axis=0)

# Use the model to predict the clear image
clear_image = model.predict(foggy_image)

# Rescale the pixel values to the original range
clear_image = clear_image * 255.0
clear_image = clear_image.astype('uint8')

# Reshape the clear image
clear_image = np.squeeze(clear_image, axis=0)

# Display the clear image
cv2.imshow("Clear Image", clear_image)
cv2.waitKey(0)
cv2.destroyAllWindows()



1/1 [==============================] - 0s 99ms/step
